## 머신러닝 응용

## 02. 앙상블 모델 - Voting

<img src = "https://images.unsplash.com/photo-1540910419892-4a36d2c3266c?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=1170&q=80" width=80% align="center"/>

<div align="right">사진: <a href="https://unsplash.com/ko/%EC%82%AC%EC%A7%84/T9CXBZLUvic?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>의<a href="https://unsplash.com/@element5digital?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Element5 Digital</a>
</div>

### 0. 데이터 준비하기
- **데이터 설명**<br>
타이타닉 데이터셋은 1912년에 발생한 타이타닉호의 침몰 사고와 관련된 정보를 담고 있는 데이터셋입니다. 이 데이터셋은 Kaggle에서 제공하며 기계 학습과 데이터 분석을 위해 널리 사용되는 예제 데이터셋 중 하나로, 생존자 예측과 관련된 문제에 대한 연구 및 분석을 수행하는 데 자주 활용됩니다. <br><br>
이번 실습에서는 해당 데이터셋의 전처리 작업(결측치 처리, 가변수화, 정규화 등)을 완료한 데이터 파일('titanic_preprocessing.csv')을 활용할 예정이며, 각 컬럼은 다음과 같은 내용을 포함하고 있습니다. 

| 컬럼명         | 설명                                      |
|----------------|--------------------------------------------|
| Survived       | 생존 여부 (1: 생존, 0: 사망)               |
| Age            | 나이                                       |
| SibSp          | 함께 탑승한 형제자매 또는 배우자의 수       |
| Parch          | 함께 탑승한 부모 또는 자녀의 수             |
| Fare           | 운임 요금                                  |
| Title_Mr       | Mr. 호칭을 갖는 탑승객 여부 (1: 해당, 0: 해당X) |
| Title_Mrs      | Mrs. 호칭을 갖는 탑승객 여부 (1: 해당, 0: 해당X)|
| Title_Others   | Mr., Mrs. Miss 이외의 호칭을 갖는 탑승객 여부  |
| Pclass_2       | 2등석 여부 (1: 해당, 0: 해당X)              |
| Pclass_3       | 3등석 여부 (1: 해당, 0: 해당X)              |
| Sex_male       | 성별이 남성인지 여부 (1: 남성, 0: 여성)     |
| Embarked_Q     | 탑승지가 Queenstown인지 여부 (1: 해당, 0: 해당X)|
| Embarked_S     | 탑승지가 Southampton인지 여부 (1: 해당, 0: 해당X)|


#### 1️⃣ 데이터 불러오기

In [1]:
# 라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 데이터 읽어오기
df = pd.read_csv("./data/titanic_preprocessing.csv")

In [3]:
# 샘플 데이터 확인하기
df.head()

,Survived,Age,SibSp,Parch,Fare,Title_Mr,Title_Mrs,Title_Others,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0.0,0.271174,0.125,0.0,0.014151,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.472229,0.125,0.0,0.139136,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.321438,0.000,0.0,0.015469,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.434531,0.125,0.0,0.103644,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.434531,0.000,0.0,0.015713,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Survived      891 non-null    float64
 1   Age           891 non-null    float64
 2   SibSp         891 non-null    float64
 3   Parch         891 non-null    float64
 4   Fare          891 non-null    float64
 5   Title_Mr      891 non-null    float64
 6   Title_Mrs     891 non-null    float64
 7   Title_Others  891 non-null    float64
 8   Pclass_2      891 non-null    float64
 9   Pclass_3      891 non-null    float64
 10  Sex_male      891 non-null    float64
 11  Embarked_Q    891 non-null    float64
 12  Embarked_S    891 non-null    float64
dtypes: float64(13)
memory usage: 90.6 KB


#### 2️⃣ 데이터 분리하기

In [5]:
# Feature는 x, Target은 y 로 저장합니다.
x = df.drop(columns=['Survived'])
y = df['Survived']

In [6]:
# sklearn 패키키 증 train_test_split 함수 불러오기
from sklearn.model_selection import train_test_split

# Feature와 Target을 train, test 데이터 셋으로 나누기
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


---

### 1. Hard Voting

#### 1️⃣ 라이브러리 호출하기

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report

#### 2️⃣ 선언하기 
① DecisionTreeClassifier, KNeighborsClassifier, SVC 클래스 객체 생성하기<br>
② VotingClassifier 클래스 객체 생성하기

In [10]:
# 다양한 단일 모델 생성
model_dt = DecisionTreeClassifier()
model_knn = KNeighborsClassifier()
model_svm = SVC()

In [11]:
# 앙상블(Voting) 모델 생성
model_voting = VotingClassifier(estimators=[('dt', model_dt), 
                                            ('knn', model_knn), 
                                            ('svm', model_svm)], 
                                voting='hard')

#### 3️⃣ 학습하기
fit 메서드로 가중치 값을 추정한다.

In [12]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('knn', KNeighborsClassifier()), ('svm', SVC())])

#### 4️⃣ 예측하기
predict 메서드로 새로운 입력 데이터에 대한 출력 데이터 예측



In [13]:
y_pred = model_voting.predict(x_test)

#### 5️⃣ 평가하기

In [14]:
print("[classification_report]")
print(classification_report(y_test, y_pred))

[classification_report]
              precision    recall  f1-score   support

         0.0       0.84      0.86      0.85       105
         1.0       0.79      0.77      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



---

### 2. Soft Voting

#### 1️⃣ 라이브러리 호출하기

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report

#### 2️⃣ 선언하기

**📌 <font color='red'>주의사항</font>**<br>
분류 모델에서 클래스에 대한 예측 확률을 기본 옵션으로 지원하지 않는 경우가 있습니다. <br>예를 들어, ***SVC() 모델의 경우 probaility 옵션을 True로 변경해 주어야 합니다.***

In [16]:
# 다양한 단일 모델 생성
model_dt = DecisionTreeClassifier()
model_knn = KNeighborsClassifier()
model_svm = SVC(probability=True)

In [17]:
# 앙상블(Voting) 모델 생성
model_voting2 = VotingClassifier(estimators=[('dt', model_dt), 
                                            ('knn', model_knn), 
                                            ('svm', model_svm)], 
                                voting='soft')

#### 3️⃣ 학습하기

In [18]:
model_voting2.fit(x_train, y_train)

VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('knn', KNeighborsClassifier()),
                             ('svm', SVC(probability=True))],
                 voting='soft')

#### 4️⃣ 예측하기

In [19]:
y_pred = model_voting2.predict(x_test)

#### 5️⃣ 평가하기

In [20]:
print("[classification_report]")
print(classification_report(y_test, y_pred))

[classification_report]
              precision    recall  f1-score   support

         0.0       0.86      0.87      0.86       105
         1.0       0.81      0.80      0.80        74

    accuracy                           0.84       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



---